In [73]:
import pandas
import random
import numpy as np
#read the csv file and only column 'guesses'
guesses = pandas.read_csv('valid_guesses.csv',usecols=['guesses'])
solutions = pandas.read_csv('valid_solutions.csv',usecols=['solutions'])

guesses = guesses.to_numpy()
solutions = solutions.to_numpy()

In [165]:
class LetterPos:
    def __init__(self, letter, pos, gpos, perfect):
        self.letter = letter
        self.pos = pos
        self.gpos = gpos
        self.perfect = perfect

class GameState:
    def __init__(self):
        self.guesses = []
        self.solution = random.choice(solutions)[0] 
        self.correct_letters = []
        self.incorrect_letters = []
        self.turn = 0

    def output_turn(self):
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                print(self.correct_letters[self.turn][i].letter)
                print(self.correct_letters[self.turn][i].pos)
                print(self.correct_letters[self.turn][i].perfect)

    def pretty_output(self):
        guess_result = list("⬛⬛⬛⬛⬛")
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                guess_result[self.correct_letters[self.turn][i].gpos] = "🟩" if self.correct_letters[self.turn][i].perfect else "🟨"
        print(guess_result)

In [166]:
def compare(guess, solution):
    #compare the two words
    guess_letters = list(guess)
    solution_letters = list(solution)
    print(solution_letters)
    print(guess_letters)
    
    
    correct_letters = []

    #check for letter matches
    for i in range(5):
        for j in range(5):
            if(guess_letters[i] == solution_letters[j]):
                guess_grade = LetterPos(guess_letters[i], j, i, (i == j))
                correct_letters.append(guess_grade)
    
    #edge cases
    #check for repeating LetterPos
    if(len(correct_letters) > 2):
        for i in range(len(correct_letters)):
            for j in range(len(correct_letters)):
                if(correct_letters[i].pos == correct_letters[j+1].pos):
                    correct_letters.pop(j)

    #win condition
    #check if the guess is perfect
    if(len(correct_letters) == 5):
        isPerfect = True
        for i in range(len(correct_letters)):
            if(correct_letters[i].perfect == False):
                isPerfect = False
        print(isPerfect)

    
    return correct_letters

    

In [167]:
#game loop
def wordlegame():
    game = GameState()
    #randomly select a word from the list solutions
    print(game.solution)

    for i in range(0, 6):
        guess = random.choice(guesses)
        guess = guess[0]
        print(guess)
        #compare the guess to the solution
        game.correct_letters.append(compare(guess, game.solution))
        game.pretty_output()
        game.turn += 1




In [168]:
wordlegame()

slang
miffs
['s', 'l', 'a', 'n', 'g']
['m', 'i', 'f', 'f', 's']
['🟨', '⬛', '⬛', '⬛', '⬛']
ofter
['s', 'l', 'a', 'n', 'g']
['o', 'f', 't', 'e', 'r']
tubas
['s', 'l', 'a', 'n', 'g']
['t', 'u', 'b', 'a', 's']
['⬛', '⬛', '🟨', '⬛', '⬛']
['🟨', '⬛', '🟨', '⬛', '⬛']
bezes
['s', 'l', 'a', 'n', 'g']
['b', 'e', 'z', 'e', 's']
['🟨', '⬛', '⬛', '⬛', '⬛']
mieve
['s', 'l', 'a', 'n', 'g']
['m', 'i', 'e', 'v', 'e']
teaed
['s', 'l', 'a', 'n', 'g']
['t', 'e', 'a', 'e', 'd']
['⬛', '⬛', '🟩', '⬛', '⬛']
